# Случайные леса
__Суммарное количество баллов: 10__

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным и сравнить его эффективность с ансамблями из самых популярных библиотек.

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import matplotlib
import copy
from typing import Tuple

Реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание. Используйте реализацию дерева из HW3.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [ ]:
from functools import reduce
from math import log

def get_p(x):
    def tmp(clazz):
        return len([ t for t in x if t == clazz]) / len(x)
    
    return tmp

def gini(x):
    classes = np.unique(x)

    p = get_p(x)

    return reduce(
        lambda acc, clazz: acc + p(clazz) * (1 - p(clazz)),
        classes,
        0
    )
    
def entropy(x):
    classes = np.unique(x)

    p = get_p(x)

    return -reduce(
        lambda acc, clazz: acc + p(clazz) * log(p(clazz)),
        classes,
        0
    )

def gain(left_y, right_y, criterion):
    return (len(left_y) * criterion(left_y) + len(right_y) * criterion(right_y)) / (len(left_y) + len(right_y))

class DecisionTreeLeaf:
    def __init__(self, labels):
        self.probabilities = {}
        unique_labels = np.unique(labels)
        
        for label in unique_labels:
            self.probabilities[label] = len([ y for y in labels if y == label ]) / len(labels)
        
        self.y = reduce(
            lambda max_label, current_label: current_label if self.probabilities[max_label] < self.probabilities[current_label] else max_label,
            self.probabilities
        )


class DecisionTreeNode:
    def __init__(self, split_dim, split_value, left, right):
        self.split_dim = split_dim
        self.split_value = split_value
        self.left = left
        self.right = right


class DecisionTreeClassifier:
    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1):
        self.root = None
        self.__gain = lambda left, right: gain(left, right, gini if criterion == 'gini' else entropy)
        self.__max_depth = max_depth
        self.__min_samples_leaf = min_samples_leaf
    
    def fit(self, X, y):
        self.__data = {} # Словарь: Ключ -- id, значение -- пара (пример, метка)
        
        for x in X: # Заполняем self.__data
            label = -1
            
            for i in y:
                if i[0] == x[0]:
                    label = i[1]
                    break
            
            if label != -1:
                self.__data[x[0]] = (x[1:], label)
        
        # Для каждой фичи определяем "границы" минимум и максимум, которого она достигает на датасете
        tmp = np.empty((len(self.__data), len(next(iter(self.__data.values()))[0]))) # Размеры tmp -- (<размер датасета>, <количество фич первого элемента словаря>)

        counter = 0;
        
        for id in self.__data:
            tmp[counter] = self.__data[id][0]
            counter += 1

        self.__borders = np.transpose(np.array([np.amin(tmp, axis=0), np.amax(tmp, axis = 0)]))

        self.root = self.__get_node_or_leaf(self.__data, 1)
                    
    def __get_node_or_leaf(self, data_subset, depth):
        labels = [ data_subset[id][1] for id in data_subset ]
        
        if len(data_subset.items()) <= self.__min_samples_leaf:
            return DecisionTreeLeaf(labels)
        
        if self.__max_depth is not None and self.__max_depth <= depth: # Дререво достигло максимальной глубины
            return DecisionTreeLeaf(labels)
        
        if len(np.unique(labels)) == 1: # Все объекты принадлежат к одному классу
            return DecisionTreeLeaf(labels)
        

        gain = None # Вместо +inf
        split_feature_idx = 0
        split_treshold = 0
        left_data_subset, right_data_subset = {}, {}
        
        for feature_id in range( len( next( iter( data_subset.values() ) )[0] ) ): # Итерируемся 0..<длина фич первого элемента словаря>

            feature_min, feature_range = \
                self.__borders[feature_id][0], \
                self.__borders[feature_id][1] - self.__borders[feature_id][0]

            for i in range(0, 101): # Делим облазть значений каждой фичи на 100 кусков и выполняем разбиения по границам этих кусков
                feature_value = feature_min + (feature_range / 100 * i)
                samples_with_labels_left, samples_with_labels_right = {}, {}

                for id in data_subset:
                    if data_subset[id][0][feature_id] < feature_value:
                        samples_with_labels_left[id] = data_subset[id]
                    else:
                        samples_with_labels_right[id] = data_subset[id]

                local_gain = self.__gain(
                    [ data_subset[id][1] for id in samples_with_labels_left ],
                    [ data_subset[id][1] for id in samples_with_labels_right ]
                )
                
                if gain is None or local_gain < gain:
                    gain = local_gain
                    split_feature_idx = feature_id
                    split_treshold = feature_value
                    left_data_subset = samples_with_labels_left.copy()
                    right_data_subset = samples_with_labels_right.copy()
        
        return DecisionTreeNode(
            split_feature_idx,
            split_treshold,
            self.__get_node_or_leaf(left_data_subset, depth + 1),
            self.__get_node_or_leaf(right_data_subset, depth + 1)
        )

    
    def predict_proba(self, X):
        result = np.empty(len(X), dtype=object)

        for sample_idx, sample in enumerate(X):
            current_node = self.root

            while type(current_node) != type(DecisionTreeLeaf([0])): # Аргумент, передоваемый в Leaf -- костыль
                split_feature_idx = current_node.split_dim + 1 # +1 потому что при обучении мы проигнорировали колонку ID, поэтому все индексы в узлах на единицу меньше

                if sample[split_feature_idx] < current_node.split_value:
                    current_node = current_node.left
                else:
                    current_node = current_node.right
            
            result[sample_idx] = current_node.probabilities
        
        return result
    
    def predict(self, X):
        proba = self.predict_proba(X)
        return [ max(p.keys(), key=lambda k: p[k]) for p in proba ]

In [ ]:
class RandomForestClassifier:
    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto", n_estimators=10):
        self.__max_features = max_features
        self.__n_estimators = n_estimators
        self.__trees = np.array([ DecisionTreeClassifier(
            criterion=criterion,
            max_depth=max_depth,
            min_samples_leaf=min_samples_leaf
        ) for _ in range(n_estimators) ])
        
    
    def fit(self, X, y):
        bagged_data = self.__get_bagged_datasets(X, y, self.__n_estimators, len(X)); # TODO how to determine number of samples in bagged dataset?
        [ self.__trees[tree_idx].fit(bagged_data[tree_idx][0], bagged_data[tree_idx][1]) for tree_idx in range(len(self.__trees)) ]
    
    def predict(self, X):
        raise NotImplementedError()

    
    def __get_bagged_datasets(self, X, y, number_of_sets, number_of_samples):
        X_sets = np.empty((number_of_sets, number_of_samples, X.shape[1]))
        y_sets = np.empty((number_of_sets, number_of_samples, y.shape[1]))
        
        for set_number in range(number_of_sets):
            indexes = np.random.choice(np.arange(X.shape[0]), number_of_samples)
            X_sets[set_number] = X[indexes]
            y_sets[set_number] = y[indexes]
        
        return [ (X_sets[idx], y_sets[idx]) for idx in range(number_of_sets) ]

### Задание 3 (2 балла)
Оптимизируйте по `AUC` на кроссвалидации (размер валидационной выборки - 20%) параметры своей реализации `Random Forest`: 

максимальную глубину деревьев из [2, 3, 5, 7, 10], количество деревьев из [5, 10, 20, 30, 50, 100]. 

Постройте `ROC` кривую (и выведите `AUC` и `accuracy`) для лучшего варианта.

Подсказка: можно построить сразу 100 деревьев глубины 10, а потом убирать деревья и
глубину.

In [ ]:
def train_test_split(X: np.array, y: np.array, ratio: float = 0.7) -> Tuple[np.array, np.array, np.array, np.array]:
    # Возвращает X_train, y_train, X_test, y_test
    # X_train и X_test - массив векторов - две части массива X, разделенного в состветсви с коэффициентом ratio
    # y_train и y_test - соответствующие X_train и X_test метки классов
    treshold: int = int(X.shape[0] * ratio)
    return X[:treshold], y[:treshold], X[treshold:], y[treshold:]

X, y = pd.read_csv('hw_trees_data/x_spam_train.csv').to_numpy(), pd.read_csv('hw_trees_data/y_spam_train.csv').to_numpy()
X_train, y_train, X_test, y_test = train_test_split(X, y, 0.1)

forest = RandomForestClassifier()
forest.fit(X_train, y_train)

Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest выглядит так:
1. Посчитать out-of-bag ошибку предсказания `err_oob` (https://en.wikipedia.org/wiki/Out-of-bag_error)
2. Перемешать значения признака `j` у объектов выборки (у каждого из объектов изменится значение признака `j` на какой-то другой)
3. Посчитать out-of-bag ошибку (`err_oob_j`) еще раз.
4. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [ ]:
def feature_importance(rfc):
    raise NotImplementedError()

def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    idicies = np.argsort(importance)[::-1][:k]
    return np.array(names)[idicies]

Протестируйте решение на простом синтетическом наборе данных. В результате должна получиться точность `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [ ]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, 
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc))

Проверьте, какие признаки важны для датасета spam? (Используйте файлы x_spam_train и y_spam_train)

_Ваш ответ_

1. Обучите модель на всех данных из x_spam_train и y_spam_train.
2. Сделайте submit своего решения и получите значение f1_score не менее 0.6

In [ ]:
submission = pd.DataFrame(columns = ["Id", "Expected"])
submission["Id"] = test["Id"]
submission["Expected"] = #YOUR CODE
submission.to_csv('submission.csv', index=False)

В качестве альтернативы попробуем библиотечные реализации ансамблей моделей. 

1. [CatBoost](https://catboost.ai/docs/)
2. [XGBoost](https://xgboost.readthedocs.io/en/latest/)
3. [LightGBM](https://lightgbm.readthedocs.io/en/latest/)


Установите необходимые библиотеки. 
Возможно, потребуется установка дополнительных пакетов.

In [ ]:
!pip install lightgbm
!pip install catboost
!pip install xgboost

1. Примените модели для нашего датасета.

2. Для стандартного набора параметров у каждой модели нарисуйте `ROC` кривую и выведите `AUC` и `accuracy`.

3. Посчитайте время обучения каждой модели (можно использовать [timeit magic](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit)).

4. Сравните метрики качества и скорость обучения моделей. Какие выводы можно сделать?

In [ ]:
# YOUR_CODE

_Ваш ответ_